In [12]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


data_dir = 'data/vegetable_images' 


transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])


train_dataset = ImageFolder(root=data_dir + '/train', transform=transform)
val_dataset = ImageFolder(root=data_dir + '/validation', transform=transform)

val_dataset_with_noise = ImageFolder(root=data_dir + '/gaussian_noise/validation/100', transform=transform)
test_dataset = ImageFolder(root=data_dir + '/gaussian_noise/test/100', transform=transform)
test_dataset_without_noise = ImageFolder(root=data_dir + '/test', transform=transform)


dataloaders = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset, batch_size=64)
               }

dataloaders_with_noise = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset_with_noise, batch_size=64)
               }


test_loader = DataLoader(test_dataset, batch_size=64)
test_loader_without_noise = DataLoader(test_dataset_without_noise, batch_size=64)




In [15]:
print(len(dataloaders['train'].dataset))
print(len(dataloaders['val'].dataset))

dataset_sizes = {'train' : len(dataloaders['train'].dataset), 'val': len(dataloaders['val'].dataset), 'test': len(test_loader.dataset)}
print(len(test_loader.dataset))
classes  = train_dataset.classes
classes

15000
3000
3000


['Bean',
 'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']

In [16]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [17]:
model = models.alexnet(pretrained=True)

In [18]:
print(model._modules['classifier'])

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)


In [19]:
in_features = model._modules['classifier'][-1].in_features
out_features = len(classes)
model._modules['classifier'][-1] = nn.Linear(in_features, out_features, bias=True)
print(model._modules['classifier'])

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=15, bias=True)
)


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

cuda


# CLEAN IMAGES WITHOUT NOISE BEFORE TRAINING

In [23]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader_without_noise:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 7.6 %
Accuracy of Bean: 15.0 %
Accuracy of Bitter_Gourd: 1.0 %
Accuracy of Bottle_Gourd: 1.0 %
Accuracy of Brinjal: 4.0 %
Accuracy of Broccoli: 1.0 %
Accuracy of Cabbage: 15.5 %
Accuracy of Capsicum: 0.0 %
Accuracy of Carrot: 1.5 %
Accuracy of Cauliflower: 1.5 %
Accuracy of Cucumber: 7.0 %


# WITH 100% NOISE BEFORE TRAINING

In [24]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 7.533333333333333 %
Accuracy of Bean: 44.5 %
Accuracy of Bitter_Gourd: 0.5 %
Accuracy of Bottle_Gourd: 1.5 %
Accuracy of Brinjal: 0.0 %
Accuracy of Broccoli: 13.0 %
Accuracy of Cabbage: 17.5 %
Accuracy of Capsicum: 4.5 %
Accuracy of Carrot: 3.0 %
Accuracy of Cauliflower: 14.5 %
Accuracy of Cucumber: 0.0 %


In [25]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

       
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs =  torch.tensor(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))


    model.load_state_dict(best_model_wts)
    return model

In [26]:
for param in model.parameters():
    param.requires_grad = True

In [27]:
criterion = nn.CrossEntropyLoss()


optimizer = optim.SGD(model.parameters(), lr=0.001)


step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=25)

Epoch 0/24
----------


/tmp/ipykernel_189093/2272575281.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs =  torch.tensor(inputs).to(device)
/tmp/ipykernel_189093/2272575281.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


train Loss: 0.5360 Acc: 0.8581
val Loss: 0.0945 Acc: 0.9740

Epoch 1/24
----------
train Loss: 0.1065 Acc: 0.9697
val Loss: 0.0501 Acc: 0.9890

Epoch 2/24
----------
train Loss: 0.0653 Acc: 0.9817
val Loss: 0.0299 Acc: 0.9947

Epoch 3/24
----------
train Loss: 0.0453 Acc: 0.9874
val Loss: 0.0240 Acc: 0.9943

Epoch 4/24
----------
train Loss: 0.0349 Acc: 0.9906
val Loss: 0.0196 Acc: 0.9943

Epoch 5/24
----------
train Loss: 0.0291 Acc: 0.9927
val Loss: 0.0173 Acc: 0.9950

Epoch 6/24
----------
train Loss: 0.0243 Acc: 0.9932
val Loss: 0.0178 Acc: 0.9953

Epoch 7/24
----------
train Loss: 0.0210 Acc: 0.9947
val Loss: 0.0159 Acc: 0.9957

Epoch 8/24
----------
train Loss: 0.0184 Acc: 0.9957
val Loss: 0.0159 Acc: 0.9953

Epoch 9/24
----------
train Loss: 0.0198 Acc: 0.9955
val Loss: 0.0156 Acc: 0.9957

Epoch 10/24
----------
train Loss: 0.0191 Acc: 0.9953
val Loss: 0.0152 Acc: 0.9957

Epoch 11/24
----------
train Loss: 0.0182 Acc: 0.9957
val Loss: 0.0150 Acc: 0.9957

Epoch 12/24
----------
t

In [70]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader_without_noise:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 99.66666666666667 %
Accuracy of Bean: 100.0 %
Accuracy of Bitter_Gourd: 99.0 %
Accuracy of Bottle_Gourd: 100.0 %
Accuracy of Brinjal: 99.5 %
Accuracy of Broccoli: 99.0 %
Accuracy of Cabbage: 99.5 %
Accuracy of Capsicum: 99.5 %
Accuracy of Carrot: 100.0 %
Accuracy of Cauliflower: 100.0 %
Accuracy of Cucumber: 99.0 %


In [71]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(len(classes))]
    n_class_samples = [0 for i in range(len(classes))]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(len(images)):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Accuracy of the network: 15.533333333333333 %
Accuracy of Bean: 0.0 %
Accuracy of Bitter_Gourd: 95.0 %
Accuracy of Bottle_Gourd: 9.5 %
Accuracy of Brinjal: 0.0 %
Accuracy of Broccoli: 27.5 %
Accuracy of Cabbage: 0.0 %
Accuracy of Capsicum: 0.0 %
Accuracy of Carrot: 0.5 %
Accuracy of Cauliflower: 0.5 %
Accuracy of Cucumber: 0.0 %
